In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model,Model
import cv2

dnn_model = tf.keras.models.load_model('new_dnn_model.h5')
dcnn_model = tf.keras.models.load_model('new_dcnn_model.h5')

def prepare(file):
    IMG_SIZE = 24  
    new_array = cv2.resize(file, (IMG_SIZE, IMG_SIZE),interpolation=cv2.INTER_AREA)  
    new_array=new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    new_array=new_array/255.0
    return new_array
def prepare_gray(file):
    IMG_SIZE = 24    
    new_array = cv2.resize(file, (IMG_SIZE, IMG_SIZE),interpolation=cv2.INTER_AREA) 
    new_array=new_array.reshape(-1, IMG_SIZE, IMG_SIZE)
    new_array=new_array/255.0
    return new_array
def predict(arr,arr2):
    state=[]
    state.append(arr[0]+arr2[0])
    state.append(arr[1]+arr2[1])
    index=np.argmax(state)
    a=state[0]/(state[0]+state[1])*100
    if a>= 50.0:
        return True
    else:
        return False

In [2]:
import numpy as np
import cv2
import time
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('frontalEyes35x16.xml')
cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while 1:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if ret == True: 
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            for (ex,ey,ew,eh) in eyes:
                cv2.rectangle(roi_color,(ex,ey),(int((ex+ew)*0.55),ey+eh),(0,255,0),1)
                #cv2.rectangle(roi_color,(int((ex+ew)*0.60),ey),(ex+ew,ey+eh),(0,255,0),1)
                img2 = roi_color[int(ey*1.25):int((ey+eh)*0.95), int(ex*1.05): int((ex+ew)*0.55)]
                img3 = roi_color[int(ey*1.05):int((ey+eh)*0.95), int((ex+ew)*0.63):int((ex+ew)*0.98)]
                prediction = dcnn_model.predict([prepare(img2)])
                prediction1 = dnn_model.predict([prepare_gray(img2)])
                result = predict(prediction[0],prediction1[0])
                if result:
                    cv2.putText(frame ,"Open", (20,50),cv2.FONT_HERSHEY_SIMPLEX ,2,(0,255,0),2, cv2.LINE_AA)
                else:
                    cv2.putText(frame ,"Closed", (20,50),cv2.FONT_HERSHEY_SIMPLEX ,2,(0,0,255),2, cv2.LINE_AA)

        cv2.imshow('img',frame)
        cv2.imshow("img2",img2)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()